In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Initialise the model
model_id = 'vinai/phobert-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained('/home4/bachpt/domain_classification/Models/saved_checkpoints/checkpoint-379600')

model.eval()
model.to('cuda')

In [ ]:
# test individual sentence

import torch.nn.functional as F

inputs = tokenizer('hôm nay trời đẹp quá nên mình tính ra trường để đi dạo. Không biết có nhiêu trai đẹp không nữa. Ước gì trường mình có nhiều bạn đẹp trai để mình có thể ngắm hàng ngày. Trai trường mình vừa xấu vừa đụt, chả bù cho anh Phan Tất Bách đẹp trai ngời ngời, thông minh.', return_tensors='pt', truncation=True, max_length=256).to('cuda')


with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
softmax_values = F.softmax(logits, dim=1)
print(softmax_values)

model.config.id2label = {0: 'Chat', 1: 'Uni', 2: 'Others'}
model.config.id2label[predicted_class_id]

In [ ]:
import pandas as pd
df = pd.read_csv('/home4/bachpt/domain_classification/Data/Test/All_test_segmented.csv')
df['pred_label'] = ''
df['softmax'] = ''

In [ ]:
import torch.nn.functional as F

df_text = df['text']
predictions = []

for datum in df_text:
    inputs = tokenizer(datum, return_tensors='pt', truncation=True, max_length=256).to('cuda')
    

    with torch.no_grad():
        logits = model(**inputs).logits

        predicted_class_id = logits.argmax().item()
        
        model.config.id2label = {0: 'Chat', 1: 'Uni', 2: 'Others'}
        model.config.id2label[predicted_class_id]

        predictions.append(predicted_class_id)
    
df['pred_label'] = predictions
df.to_csv('/home4/bachpt/domain_classification/Pho_Bert_pred.csv', index=False)

In [3]:
import pandas as pd
pred_df = pd.read_csv('/home4/bachpt/domain_classification/Pho_Bert_pred.csv')
print(pred_df['pred_label'].unique())
print(pred_df['label'].unique())

[2 1 0]
['Others' 'Uni' 'Chat']
